In [1]:
import pandas as pd
import json
from utils.utils import preprocessing, catboost_learn
from catboost import CatBoostRegressor

In [2]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', None)

In [14]:
data = pd.read_excel("Data/пп_станции.xlsx")
data.shape

(322, 97)

In [4]:
data = preprocessing(data)
data.head()

/Users/alexander/Учёба/ХАКАТОНЫ/Nuclear Hack/nuclear-hack/utils/utils.py:336: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Станция'] = df['Станция'].replace(rename_dict)


,Станция,Номер линии,Линия,2024-01-01 00:00:00,2024-01-02 00:00,2024-01-03 00:00,2024-01-04 00:00,2024-01-05 00:00,2024-01-06 00:00,2024-01-07 00:00,2024-01-08 00:00,2024-01-09 00:00,2024-01-10 00:00,2024-01-11 00:00,2024-01-12 00:00,2024-01-13 00:00,2024-01-14 00:00,2024-01-15 00:00,2024-01-16 00:00,2024-01-17 00:00,2024-01-18 00:00,2024-01-19 00:00,2024-01-20 00:00,2024-01-21 00:00,2024-01-22 00:00,2024-01-23 00:00,2024-01-24 00:00,2024-01-25 00:00,2024-01-26 00:00,2024-01-27 00:00,2024-01-28 00:00,2024-01-29 00:00,2024-01-30 00:00,2024-01-31 00:00,2024-02-01 00:00,2024-02-02 00:00,2024-02-03 00:00,2024-02-04 00:00,2024-02-05 00:00,2024-02-06 00:00,2024-02-07 00:00,2024-02-08 00:00,2024-02-09 00:00,2024-02-10 00:00,2024-02-11 00:00,2024-02-12 00:00,2024-02-13 00:00,2024-02-14 00:00,2024-02-15 00:00,2024-02-16 00:00,2024-02-17 00:00,2024-02-18 00:00,2024-02-19 00:00,2024-02-20 00:00,2024-02-21 00:00,2024-02-22 00:00,2024-02-23 00:00,2024-02-24 00:00,2024-02-25 00:00,2024-02-26 00:00,2024-02-27 00:00,2024-02-28 00:00,2024-02-29 00:00,2024-03-01 00:00,2024-03-02 00:00,2024-03-03 00:00,2024-03-04 00:00,2024-03-05 00:00,2024-03-06 00:00,2024-03-07 00:00,2024-03-08 00:00,2024-03-09 00:00,2024-03-10 00:00,2024-03-11 00:00,2024-03-12 00:00,2024-03-13 00:00,2024-03-14 00:00,2024-03-15 00:00,2024-03-16 00:00,2024-03-17 00:00,2024-03-18 00:00,2024-03-19 00:00,2024-03-20 00:00,2024-03-21 00:00,2024-03-22 00:00,2024-03-23 00:00,2024-03-24 00:00,2024-03-25 00:00,2024-03-26 00:00,2024-03-27 00:00,2024-03-28 00:00,2024-03-29 00:00,2024-03-30 00:00,2024-03-31 00:00,2024-04-01 00:00,2024-04-02 00:00,2024-04-03 00:00
0,Бульвар Рокоссовского,8,СОКОЛЬНИЧЕСКАЯ,4835,2722,11209,2561,13714,15740,5816,3812,4669,7280,18189,10499,14355,16665,17944,7645,19898,10798,1510,3061,10265,18924,7410,14185,2600,14351,1223,19395,3369,12204,4205,7277,17464,18317,3697,10858,4153,9734,9018,16187,14410,18087,4186,1248,18355,13991,12338,11240,18150,1888,13882,18346,8652,6214,10012,19224,5146,11670,2812,2239,17447,5575,6368,18694,5938,11985,5490,1088,2673,7450,17639,8065,3261,8482,19679,17125,5619,4943,18651,1041,6895,2135,1229,9609,7792,14831,12097,17624,1217,12742,13998,2905,15113,16671
1,Черкизовская,8,СОКОЛЬНИЧЕСКАЯ,15638,3999,7029,15991,6496,6924,11353,3663,5061,13486,17700,3849,9090,17947,15416,7645,12877,6468,1828,12029,5797,1667,11854,13333,2485,10935,11171,16820,17624,18868,14577,11779,2947,12251,12435,14153,14294,18639,17086,12248,16338,9768,12995,16393,15487,12324,3176,15248,17010,7515,8765,3865,3025,17570,5340,12789,4556,5213,14542,15752,2181,8816,18700,4520,16592,17510,8008,12030,6452,15338,16764,14293,4148,4426,1294,17656,3129,7277,6929,2588,18593,14576,2339,15379,17814,12372,13030,6623,9721,10015,14864,7991,5441,6248
2,Преображенская площадь,8,СОКОЛЬНИЧЕСКАЯ,17117,17068,5004,2811,8365,13309,15090,13281,9745,8176,4361,19574,9135,12682,10740,18122,5698,18196,12490,10657,8169,14017,10134,7735,7813,1530,8801,15152,14727,17579,15896,10516,17054,16221,11412,3417,12981,16351,6843,6404,19921,9890,15279,9522,6083,1634,19907,4526,16736,10830,4682,6349,3033,2127,11525,17036,1276,6590,8303,11912,19336,8814,10587,9775,8939,7241,12822,9040,14517,13476,9286,17924,7788,2167,7095,6267,14169,5643,7694,12367,3391,12584,14975,19378,2349,11940,13961,6852,10961,4001,7265,6556,1132,14094
3,Сокольники,8,СОКОЛЬНИЧЕСКАЯ,5465,14176,15017,5573,18759,15545,15972,12049,14001,3637,7890,7326,8669,12214,7969,19236,11864,18937,15438,16002,2690,6518,10741,15963,3279,9381,16742,7327,7977,2009,9385,4598,2104,17897,9050,1465,11321,19227,17534,1662,19425,6400,1272,18973,11326,19224,12939,10608,3537,16012,6686,11184,10014,17931,19848,5646,6320,9933,12909,2236,10224,13928,18196,19996,13541,7822,11014,8019,7845,4898,11842,7779,18654,16816,13672,13495,9769,8076,1348,8283,16050,19168,13304,17958,19196,3010,6793,15041,12051,9446,6409,6488,10083,18651
4,Красносельская,8,СОКОЛЬНИЧЕСКАЯ,1727,19561,11373,8936,7746,4581,16152,9321,7615,14194,8012,17303,3529,7490,7338,8814,7065,11939,17022,8202,1210,2836

In [10]:
from psycopg import connect
from psycopg.rows import dict_row

def get_connection():

    POSTGRES_HOST='80.87.107.22'
    POSTGRES_PORT=5432
    POSTGRES_DATABASE='hack'
    POSTGRES_USERNAME='user'
    POSTGRES_PASSWORD='password'

    host = POSTGRES_HOST
    port = POSTGRES_PORT
    dbname = POSTGRES_DATABASE
    user = POSTGRES_USERNAME
    password = POSTGRES_PASSWORD

    cnn = connect(
        host=host,
        port=port,
        dbname=dbname,
        user=user,
        password=password,
        row_factory=dict_row
    )

    return cnn
"""
with get_connection() as cnn:
        with cnn.cursor() as cur:
            cur.execute("SELECT * FROM raw")
            data = cur.fetchall()   
            
            data = pd.DataFrame(data)
"""


    
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

default = X.iloc[:, :3]
changeable = X.iloc[:, 4:]
changeable.columns = range(len(changeable.columns))

X = pd.concat([default, changeable], axis=1)

model = CatBoostRegressor(iterations=100, depth=10, learning_rate=0.1, loss_function='RMSE', verbose=0, cat_features=default.columns.values)
model.fit(X, y)

changeable.drop(0, axis=1, inplace=True)
new_value = model.predict(X)

changeable.columns = range(len(changeable.columns))
changeable[len(changeable.columns)] = new_value

X = pd.concat([default, changeable], axis=1)

X = X[(X["Станция"] == "Красные ворота")&(X["Номер линии"] == 8)&(X["Линия"] == "СОКОЛЬНИЧЕСКАЯ")]
model.predict(X)


#model.predict(X)

array([11299.07494313])